In [22]:
import os
import cobra
import reframed
from reframed.core.environment import Environment
from reframed.cobra.simulation import FBA
from reframed.solvers import solver_instance
from reframed.solvers.solution import Status
from warnings import warn

path = '/mnt/Local_Disk_1/2_Hospital_Microbiome/Data/Modeling/Models/'

In [23]:
def auxotrophies(model, constraints=None, min_rel_growth=1e-2, min_abs_growth=1e-5):

    solver = solver_instance(model)

    if not constraints:
        constraints = Environment.complete(model, inplace=False)

    sol = FBA(model, constraints=constraints, solver=solver, get_values=False)

    if sol.status != Status.OPTIMAL or sol.fobj < min_abs_growth:
        warn("Organism does not grow in given medium.")
        return
    else:
        ref_growth = sol.fobj

    auxo = []

    for r_id in model.get_exchange_reactions():
        tmp_lb, tmp_ub = constraints[r_id]
        constraints[r_id] = (0, tmp_ub)

        sol = FBA(model, constraints=constraints, solver=solver, get_values=False)
        if sol.fobj < min_abs_growth or sol.fobj < min_rel_growth * ref_growth:
            auxo.append(r_id)

        constraints[r_id] = (tmp_lb, tmp_ub)

    return auxo

In [24]:
# model auxotrophy list
aux_dict = {}

for models in os.listdir(path):
    if models.endswith('.xml'):
        model = reframed.load_cbmodel(path + models)

        # Auxotrophy
        aux = auxotrophies(model)

        # Store auxotrophies
        aux_dict[models] = aux
        